In [21]:
from langchain_community.document_loaders import PyPDFLoader

directory = "C:\\Sowmya\\Personal\\LANGCHAIN\\ChatBot_with_Langchain_Pinecone\\data\\Canada.pdf"

def load_data(directory):
    loader = PyPDFLoader(directory)
    documents = loader.load()
    return documents

documents = load_data(directory)
len(documents)

65

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=500, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

511


In [23]:
print(docs[7].page_content)

• Total
  $2.472 trillion[5]
(16th)
• Per capita
  $60,495[5] (28th)
GDP (nominal) 2024 estimate
• Total
  $2.242 trillion[5]
(10th)
• Per capita
  $54,866[5] (18th)
Gini (2018)
  30.3[6]
medium
HDI (2022)
  0.935[7]
very high (18th)
Currency Canadian dollar ($)
(CAD)
Time zone UTC−3.5 to −8
• Summer (DST) UTC−2.5 to −7equality. It is one of the world's most ethnically diverse
and multicultural nations, the produc t of large-scale
immigration. Canada's long and complex relationship


In [24]:
# from langchain_community.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# client = AzureOpenAI(
#     api_version="2024-02-15-preview",
#     azure_deployment="atttestgpt35turbo",
#     azure_endpoint="https://cb-att-openai-instance.openai.azure.com/",
#     api_key=AZURE_OPENAI_KEY
# )

import os
from dotenv import load_dotenv
from langchain_community.embeddings import AzureOpenAIEmbeddings

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

embeddings = AzureOpenAIEmbeddings(
    deployment='vector-search-instance2',
    model='text-embedding-ada-002',
    openai_api_type='azure',
    azure_endpoint='https://cb-att-openai-instance.openai.azure.com/',
    api_key=AZURE_OPENAI_API_KEY,
)

In [25]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [26]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

Pinecone(
    api_key=PINECONE_API_KEY,
    environment='us-east-1'
)

index_name = 'langchain-chatbot'

vectordb = PineconeVectorStore.from_documents(documents=docs, embedding=embeddings, index_name=index_name)

c:\Sowmya\Personal\LANGCHAIN\ChatBot_with_Langchain_Pinecone\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [27]:
def get_similar_docs(query, k=1, score=False):
    if score:
        similar_docs = vectordb.similarity_search_with_score(query, k=k)  
    else:
        similar_docs = vectordb.similarity_search(query, k=k)
    return similar_docs

query = "What is population in Canada?"
similar_docs = get_similar_docs(query)
similar_docs

[Document(page_content="tion_and_demography/40-million). Statistics Canada. June 16, 2023. Retrieved\nSeptember 7, 2023.\n326. Edmonston, Barry; Fong, Eric (2011). The Changing Canadian Population (https://books.goo\ngle.com/books?id=VVYOgvFPvBEC&pg=PA181). McGill-Queen's University Press. p. 181.\nISBN 978-0-7735-3793-4.\n327. Zimmerman, Karla (2008). Canada (https://books.google.com/books?id=kv4nlSWLT8UC&pg\n=PA51) (10th ed.). Lonely Planet. p. 51. ISBN 978-1-74104-571-0.", metadata={'page': 50.0, 'source': 'C:\\Sowmya\\Personal\\LANGCHAIN\\ChatBot_with_Langchain_Pinecone\\data\\Canada.pdf'})]

============================= END===============================================

In [52]:
from pinecone import Pinecone

pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment = 'us-east-1'
)

In [53]:
index = pc.Index('langchain-chatbot')


In [72]:
def find_match(input):
    input_embed = embeddings.embed_query(input)
    result = index.query(vector=input_embed, top_k=2, includeMetadata=True)
    return result
    # return result['matches'][0]['metadata']['text']+"\n"+result['matches'][1]['metadata']['text']

In [73]:
result = find_match('What is population in Canada?')
result

{'matches': [{'id': '664f1f7b-dd09-42ba-b31e-24ea5afffa80',
              'metadata': {'page': 50.0,
                           'source': 'C:\\Sowmya\\Personal\\LANGCHAIN\\ChatBot_with_Langchain_Pinecone\\data\\Canada.pdf',
                           'text': 'tion_and_demography/40-million). '
                                   'Statistics Canada. June 16, 2023. '
                                   'Retrieved\n'
                                   'September 7, 2023.\n'
                                   '326. Edmonston, Barry; Fong, Eric (2011). '
                                   'The Changing Canadian Population '
                                   '(https://books.goo\n'
                                   'gle.com/books?id=VVYOgvFPvBEC&pg=PA181). '
                                   "McGill-Queen's University Press. p. 181.\n"
                                   'ISBN 978-0-7735-3793-4.\n'
                                   '327. Zimmerman, Karla (2008). Canada '
                    

In [97]:
conversation = """ 
Human: What is the economy in canada?
BOT: The economy in canada is 3 trillion per year
"""

In [98]:
query = "what is it's culture"

In [99]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2023-09-15-preview",
  azure_deployment="atttestgpt35turbo",
  azure_endpoint = 'https://cb-att-openai-instance.openai.azure.com/'
)

response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpfull assistant"},
        {"role": "user", "content": f"Given the following user query and conversation log, formulate a question that \
        would be the most relevant to provide the user with an answer from the knowledge base. \
            \n\n CONVERSATION LOG: \n{conversation}\n\nQuery: {query}\n\nRefined Query:"}
    ]
)

In [100]:
print(response.choices[0].message.content)

What are some cultural aspects of Canada?


In [101]:
print(response)

ChatCompletion(id='chatcmpl-9Ffodm7xoWYzn0gkmt4LNEEQBmpBu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='What are some cultural aspects of Canada?', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1713522687, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=92, total_tokens=100), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [102]:
items = ['book', 'pen', 'pensil', 'rubber', 'box', 'calculator']

In [109]:
conversation = ""
for i in range(len(items)-1):
    conversation += "Human: " + items[i] + "\n"
    conversation += "Bot: " + items[i+1] + "\n"

print(conversation)

Human: book
Bot: pen
Human: pen
Bot: pensil
Human: pensil
Bot: rubber
Human: rubber
Bot: box
Human: box
Bot: calculator



In [114]:
pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment = 'us-east-1'
)

index = pc.Index('langchain-chatbot')
# index = 'langchain-chatbot'
print("========================", index)

======================== <pinecone.data.index.Index object at 0x00000159204BF7C0>
